In [ ]:
from sklearn.metrics import auc, precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

def specificity(y_ture,y_pred):
    MCM = confusion_matrix(y_ture, y_pred)
    tn_sum = MCM[0, 0]
    fp_sum = MCM[0, 1]

    tp_sum = MCM[1, 1]
    fn_sum = MCM[1, 0]

    Condition_negative = tn_sum + fp_sum

    Specificity = tn_sum / Condition_negative

    return Specificity

In [4]:
import numpy as np
import pandas
import pickle
import random
from sklearn.linear_model import LogisticRegressionCV
import pickle as pl 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score,make_scorer
from sklearn.linear_model import LogisticRegressionCV
import numpy as np
import pandas
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

#build the series of talent prediction models

file_afterlabel = pandas.read_excel("The Shanghai data with labels.xlsx")


file1_1_afterlabel = file_afterlabel.loc[file_afterlabel["Sex"]==0]#select the female
file2_1_afterlabel = file_afterlabel.loc[file_afterlabel["Sex"]==1]#select the male



file1_1_afterlabel_years = [file1_1_afterlabel.loc[round(file1_1_afterlabel["Age"]) == i] for i in range(10,19)]
file2_1_afterlabel_years = [file2_1_afterlabel.loc[round(file2_1_afterlabel["Age"]) == i] for i in range(10,19)]
#oversampling
for k in range(9):
    x_lin = file1_1_afterlabel_years[k].loc[file1_1_afterlabel_years[k]["Label"]==1]
    num_range = x_lin.shape[0]
    x_lin.index = range(num_range)
    file1_1_afterlabel_years[k] = file1_1_afterlabel_years[k].loc[file1_1_afterlabel_years[k]["Label"]==0]
    num_oversampling = file1_1_afterlabel_years[k].loc[file1_1_afterlabel_years[k]["Label"]==0].shape[0]
    for i in range(num_oversampling):
        x_lin_lin = x_lin.loc[np.round(random.uniform(0,num_range-1))]
        file1_1_afterlabel_years[k] = pandas.concat([file1_1_afterlabel_years[k],pandas.DataFrame(x_lin_lin).T],axis=0)    

for k in range(9):
    x_lin = file2_1_afterlabel_years[k].loc[file2_1_afterlabel_years[k]["Label"]==1]
    num_range = x_lin.shape[0]
    x_lin.index = range(num_range)
    file2_1_afterlabel_years[k] = file2_1_afterlabel_years[k].loc[file2_1_afterlabel_years[k]["Label"]==0]
    num_oversampling = file2_1_afterlabel_years[k].loc[file2_1_afterlabel_years[k]["Label"]==0].shape[0]
    for i in range(num_oversampling):
        x_lin_lin = x_lin.loc[np.round(random.uniform(0,num_range-1))]
        file2_1_afterlabel_years[k] = pandas.concat([file2_1_afterlabel_years[k],pandas.DataFrame(x_lin_lin).T],axis=0)    

#re-order
for k in range(9):
    file1_1_afterlabel_years[k].index = range(file1_1_afterlabel_years[k].shape[0])
    file2_1_afterlabel_years[k].index = range(file2_1_afterlabel_years[k].shape[0])
    
    
X_12_0_new_afterlabel_years = [file1_1_afterlabel_years[i].loc[:][file1_1_afterlabel_years[i].columns[1:23]] for i in range(9)]
X_12_1_new_afterlabel_years = [file2_1_afterlabel_years[i].loc[:][file2_1_afterlabel_years[i].columns[1:23]] for i in range(9)]



y_12_0_new_afterlabel_years = [file1_1_afterlabel_years[i].loc[:]["Lable"]for i in range(9)]
y_12_1_new_afterlabel_years = [file2_1_afterlabel_years[i].loc[:]["Lable"]for i in range(9)]

#construct the empty list
X_12_0_new_afterlabel_train = [[] for i in range(9)]  
X_12_0_new_afterlabel_test  = [[] for i in range(9)]
y_12_0_new_afterlabel_train = [[] for i in range(9)]
y_12_0_new_afterlabel_test = [[] for i in range(9)]

X_12_1_new_afterlabel_train = [[] for i in range(9)]  
X_12_1_new_afterlabel_test  = [[] for i in range(9)]
y_12_1_new_afterlabel_train = [[] for i in range(9)]
y_12_1_new_afterlabel_test = [[] for i in range(9)]

for i in range(9):
    X_12_0_new_afterlabel_train[i],X_12_0_new_afterlabel_test[i],y_12_0_new_afterlabel_train[i],y_12_0_new_afterlabel_test[i] = train_test_split(
    X_12_0_new_afterlabel_years[i],y_12_0_new_afterlabel_years[i],test_size=0.3,random_state=2022,stratify=y_12_0_new_afterlabel_years[i])


for i in range(9):
    X_12_1_new_afterlabel_train[i],X_12_1_new_afterlabel_test[i],y_12_1_new_afterlabel_train[i],y_12_1_new_afterlabel_test[i] = train_test_split(
    X_12_1_new_afterlabel_years[i],y_12_1_new_afterlabel_years[i],test_size=0.3,random_state=2022,stratify=y_12_1_new_afterlabel_years[i])


log_10_0 = LogisticRegressionCV(scoring=make_scorer(f1_score),refit=True,penalty = "l1",solver='liblinear',max_iter=10000,cv = 5,class_weight="balanced",n_jobs=3)
log_10_1 = LogisticRegressionCV(scoring=make_scorer(f1_score),refit=True,penalty = "l1",solver='liblinear',max_iter=10000,cv = 5,class_weight="balanced",n_jobs=3)
log_11_0 = LogisticRegressionCV(scoring=make_scorer(f1_score),refit=True,penalty = "l1",solver='liblinear',max_iter=10000,cv = 5,class_weight="balanced",n_jobs=3)
log_11_1 = LogisticRegressionCV(scoring=make_scorer(f1_score),refit=True,penalty = "l1",solver='liblinear',max_iter=10000,cv = 5,class_weight="balanced",n_jobs=3)
log_12_0 = LogisticRegressionCV(scoring=make_scorer(f1_score),refit=True,penalty = "l1",solver='liblinear',max_iter=10000,cv = 5,class_weight="balanced",n_jobs=3)
log_12_1 = LogisticRegressionCV(scoring=make_scorer(f1_score),refit=True,penalty = "l1",solver='liblinear',max_iter=10000,cv = 5,class_weight="balanced",n_jobs=3)
log_13_0 = LogisticRegressionCV(scoring=make_scorer(f1_score),refit=True,penalty = "l1",solver='liblinear',max_iter=10000,cv = 5,class_weight="balanced",n_jobs=3)
log_13_1 = LogisticRegressionCV(scoring=make_scorer(f1_score),refit=True,penalty = "l1",solver='liblinear',max_iter=10000,cv = 5,class_weight="balanced",n_jobs=3)
log_14_0 = LogisticRegressionCV(scoring=make_scorer(f1_score),refit=True,penalty = "l1",solver='liblinear',max_iter=10000,cv = 5,class_weight="balanced",n_jobs=3)
log_14_1 = LogisticRegressionCV(scoring=make_scorer(f1_score),refit=True,penalty = "l1",solver='liblinear',max_iter=10000,cv = 5,class_weight="balanced",n_jobs=3)
log_15_0 = LogisticRegressionCV(scoring=make_scorer(f1_score),refit=True,penalty = "l1",solver='liblinear',max_iter=10000,cv = 5,class_weight="balanced",n_jobs=3)
log_15_1 = LogisticRegressionCV(scoring=make_scorer(f1_score),refit=True,penalty = "l1",solver='liblinear',max_iter=10000,cv = 5,class_weight="balanced",n_jobs=3)
log_16_0 = LogisticRegressionCV(scoring=make_scorer(f1_score),refit=True,penalty = "l1",solver='liblinear',max_iter=10000,cv = 5,class_weight="balanced",n_jobs=3)
log_16_1 = LogisticRegressionCV(scoring=make_scorer(f1_score),refit=True,penalty = "l1",solver='liblinear',max_iter=10000,cv = 5,class_weight="balanced",n_jobs=3)
log_17_0 = LogisticRegressionCV(scoring=make_scorer(f1_score),refit=True,penalty = "l1",solver='liblinear',max_iter=10000,cv = 5,class_weight="balanced",n_jobs=3)
log_17_1 = LogisticRegressionCV(scoring=make_scorer(f1_score),refit=True,penalty = "l1",solver='liblinear',max_iter=10000,cv = 5,class_weight="balanced",n_jobs=3)
log_18_0 = LogisticRegressionCV(scoring=make_scorer(f1_score),refit=True,penalty = "l1",solver='liblinear',max_iter=10000,cv = 5,class_weight="balanced",n_jobs=3)
log_18_1 = LogisticRegressionCV(scoring=make_scorer(f1_score),refit=True,penalty = "l1",solver='liblinear',max_iter=10000,cv = 5,class_weight="balanced",n_jobs=3)





#train the series of models
log_10_0.fit(X_12_0_new_afterlabel_train[0],y_12_0_new_afterlabel_train[0].astype("int"))
log_10_1.fit(X_12_1_new_afterlabel_train[0],y_12_1_new_afterlabel_train[0].astype("int"))

log_11_0.fit(X_12_0_new_afterlabel_train[1],y_12_0_new_afterlabel_train[1].astype("int"))
log_11_1.fit(X_12_1_new_afterlabel_train[1],y_12_1_new_afterlabel_train[1].astype("int"))

log_12_0.fit(X_12_0_new_afterlabel_train[2],y_12_0_new_afterlabel_train[2].astype("int"))
log_12_1.fit(X_12_1_new_afterlabel_train[2],y_12_1_new_afterlabel_train[2].astype("int"))

log_13_0.fit(X_12_0_new_afterlabel_train[3],y_12_0_new_afterlabel_train[3].astype("int"))
log_13_1.fit(X_12_1_new_afterlabel_train[3],y_12_1_new_afterlabel_train[3].astype("int"))

log_14_0.fit(X_12_0_new_afterlabel_train[4],y_12_0_new_afterlabel_train[4].astype("int"))
log_14_1.fit(X_12_1_new_afterlabel_train[4],y_12_1_new_afterlabel_train[4].astype("int"))

log_15_0.fit(X_12_0_new_afterlabel_train[5],y_12_0_new_afterlabel_train[5].astype("int"))
log_15_1.fit(X_12_1_new_afterlabel_train[5],y_12_1_new_afterlabel_train[5].astype("int"))

log_16_0.fit(X_12_0_new_afterlabel_train[6],y_12_0_new_afterlabel_train[6].astype("int"))
log_16_1.fit(X_12_1_new_afterlabel_train[6],y_12_1_new_afterlabel_train[6].astype("int"))

log_17_0.fit(X_12_0_new_afterlabel_train[7],y_12_0_new_afterlabel_train[7].astype("int"))
log_17_1.fit(X_12_1_new_afterlabel_train[7],y_12_1_new_afterlabel_train[7].astype("int"))

log_18_0.fit(X_12_0_new_afterlabel_train[8],y_12_0_new_afterlabel_train[8].astype("int"))
log_18_1.fit(X_12_1_new_afterlabel_train[8],y_12_1_new_afterlabel_train[8].astype("int"))


with open('log_10_0.pkl', 'wb') as f:
    pickle.dump(log_10_0, f)
with open('log_10_1.pkl', 'wb') as f:
    pickle.dump(log_10_1, f)
    
with open('log_11_0.pkl', 'wb') as f:
    pickle.dump(log_11_0, f)
with open('log_11_1.pkl', 'wb') as f:
    pickle.dump(log_11_1, f)

with open('log_12_0.pkl', 'wb') as f:
    pickle.dump(log_12_0, f)
with open('log_12_1.pkl', 'wb') as f:
    pickle.dump(log_12_1, f)
    
with open('log_13_0.pkl', 'wb') as f:
    pickle.dump(log_13_0, f)
with open('log_13_1.pkl', 'wb') as f:
    pickle.dump(log_13_1, f)
    
with open('log_14_0.pkl', 'wb') as f:
    pickle.dump(log_14_0, f)
with open('log_14_1.pkl', 'wb') as f:
    pickle.dump(log_14_1, f)
    
with open('log_15_0.pkl', 'wb') as f:
    pickle.dump(log_15_0, f)
with open('log_15_1.pkl', 'wb') as f:
    pickle.dump(log_15_1, f)
    
with open('log_16_0.pkl', 'wb') as f:
    pickle.dump(log_16_0, f)
with open('log_16_1.pkl', 'wb') as f:
    pickle.dump(log_16_1, f)
    
with open('log_17_0.pkl', 'wb') as f:
    pickle.dump(log_17_0, f)
with open('log_17_1.pkl', 'wb') as f:
    pickle.dump(log_17_1, f)
    
with open('log_18_0.pkl', 'wb') as f:
    pickle.dump(log_18_0, f)
with open('log_18_1.pkl', 'wb') as f:
    pickle.dump(log_18_1, f)


In [8]:
import numpy as np
import pandas
import pickle
import random
from sklearn.linear_model import LogisticRegressionCV
import pickle as pl 
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


file_afterlabel = pandas.read_excel("The Shanghai data with labels.xlsx")


file1_1_afterlabel = file_afterlabel.loc[file_afterlabel["Sex"]==0]#select the female
file2_1_afterlabel = file_afterlabel.loc[file_afterlabel["Sex"]==1]#select the male

#进行oversampling
for k in range(9):
    x_lin = file1_1_afterlabel_years[k].loc[file1_1_afterlabel_years[k]["Label"]==1]
    num_range = x_lin.shape[0]
    x_lin.index = range(num_range)
    file1_1_afterlabel_years[k] = file1_1_afterlabel_years[k].loc[file1_1_afterlabel_years[k]["Label"]==0]
    num_oversampling = file1_1_afterlabel_years[k].loc[file1_1_afterlabel_years[k]["Label"]==0].shape[0]
    for i in range(num_oversampling):
        x_lin_lin = x_lin.loc[np.round(random.uniform(0,num_range-1))]
        file1_1_afterlabel_years[k] = pandas.concat([file1_1_afterlabel_years[k],pandas.DataFrame(x_lin_lin).T],axis=0)    

for k in range(9):
    x_lin = file2_1_afterlabel_years[k].loc[file2_1_afterlabel_years[k]["Label"]==1]
    num_range = x_lin.shape[0]
    x_lin.index = range(num_range)
    file2_1_afterlabel_years[k] = file2_1_afterlabel_years[k].loc[file2_1_afterlabel_years[k]["Label"]==0]
    num_oversampling = file2_1_afterlabel_years[k].loc[file2_1_afterlabel_years[k]["Label"]==0].shape[0]
    for i in range(num_oversampling):
        x_lin_lin = x_lin.loc[np.round(random.uniform(0,num_range-1))]
        file2_1_afterlabel_years[k] = pandas.concat([file2_1_afterlabel_years[k],pandas.DataFrame(x_lin_lin).T],axis=0)    

#re-order
for k in range(9):
    file1_1_afterlabel_years[k].index = range(file1_1_afterlabel_years[k].shape[0])
    file2_1_afterlabel_years[k].index = range(file2_1_afterlabel_years[k].shape[0])

index_name = file1_1_afterlabel.columns[1:23]

file1_1_afterlabel_years = [file1_1_afterlabel.loc[round(file1_1_afterlabel["Age"]) == i] for i in range(10,19)]#分别选出来10-18岁
file2_1_afterlabel_years = [file2_1_afterlabel.loc[round(file2_1_afterlabel["Age"]) == i] for i in range(10,19)]

X_12_0_new_afterlabel_years = [file1_1_afterlabel_years[i].loc[:][file1_1_afterlabel_years[i].columns[1:23]] for i in range(9)]
X_12_1_new_afterlabel_years = [file2_1_afterlabel_years[i].loc[:][file2_1_afterlabel_years[i].columns[1:23]] for i in range(9)]



y_12_0_new_afterlabel_years = [file1_1_afterlabel_years[i].loc[:]["Label"]for i in range(9)]
y_12_1_new_afterlabel_years = [file2_1_afterlabel_years[i].loc[:]["Label"]for i in range(9)]

X_12_0_new_afterlabel_train = [[] for i in range(9)]  
X_12_0_new_afterlabel_test  = [[] for i in range(9)]
y_12_0_new_afterlabel_train = [[] for i in range(9)]
y_12_0_new_afterlabel_test = [[] for i in range(9)]

X_12_1_new_afterlabel_train = [[] for i in range(9)]  
X_12_1_new_afterlabel_test  = [[] for i in range(9)]
y_12_1_new_afterlabel_train = [[] for i in range(9)]
y_12_1_new_afterlabel_test = [[] for i in range(9)]
#set random seed
for i in range(9):
    X_12_0_new_afterlabel_train[i],X_12_0_new_afterlabel_test[i],y_12_0_new_afterlabel_train[i],y_12_0_new_afterlabel_test[i] = train_test_split(
    X_12_0_new_afterlabel_years[i],y_12_0_new_afterlabel_years[i],test_size=0.3,random_state=2022,stratify=y_12_0_new_afterlabel_years[i])


for i in range(9):
    X_12_1_new_afterlabel_train[i],X_12_1_new_afterlabel_test[i],y_12_1_new_afterlabel_train[i],y_12_1_new_afterlabel_test[i] = train_test_split(
    X_12_1_new_afterlabel_years[i],y_12_1_new_afterlabel_years[i],test_size=0.3,random_state=2022,stratify=y_12_1_new_afterlabel_years[i])



y = [[] for i in range(9)]
y_1 = [[] for i in range(9)]

proba_test = [[] for i in range(9)]
proba_test_1 = [[] for i in range(9)]

y_pred = [[] for i in range(9)]
y_pred_1 = [[] for i in range(9)]
for i in range(9):
    if i == 0:
        y[i] = y_12_0_new_afterlabel_test[i]
        y_pred[i] = log_10_0.predict(X_12_0_new_afterlabel_test[i])
        proba_test[i] = log_10_0.predict_proba(X_12_0_new_afterlabel_test[i])
        
        y_1[i] = y_12_1_new_afterlabel_test[i]
        y_pred_1[i] = log_10_1.predict(X_12_1_new_afterlabel_test[i])
        proba_test_1[i] = log_10_1.predict_proba(X_12_1_new_afterlabel_test[i])
    if i == 1:
        y[i] = y_12_0_new_afterlabel_test[i]
        y_pred[i] = log_11_0.predict(X_12_0_new_afterlabel_test[i])
        proba_test[i] = log_11_0.predict_proba(X_12_0_new_afterlabel_test[i])        
        
        y_1[i] = y_12_1_new_afterlabel_test[i]
        y_pred_1[i] = log_11_1.predict(X_12_1_new_afterlabel_test[i])
        proba_test_1[i] = log_11_1.predict_proba(X_12_1_new_afterlabel_test[i])
    if i == 2:
        y[i] = y_12_0_new_afterlabel_test[i]
        y_pred[i] = log_12_0.predict(X_12_0_new_afterlabel_test[i])
        proba_test[i] = log_12_0.predict_proba(X_12_0_new_afterlabel_test[i])               
        
        y_1[i] = y_12_1_new_afterlabel_test[i]
        y_pred_1[i] = log_12_1.predict(X_12_1_new_afterlabel_test[i])
        proba_test_1[i] = log_12_1.predict_proba(X_12_1_new_afterlabel_test[i])
    if i == 3:
        y[i] = y_12_0_new_afterlabel_test[i]
        y_pred[i] = log_13_0.predict(X_12_0_new_afterlabel_test[i])
        proba_test[i] = log_13_0.predict_proba(X_12_0_new_afterlabel_test[i])            
        
        y_1[i] = y_12_1_new_afterlabel_test[i]
        y_pred_1[i] = log_13_1.predict(X_12_1_new_afterlabel_test[i])
        proba_test_1[i] = log_13_1.predict_proba(X_12_1_new_afterlabel_test[i])        
    if i == 4:
        y[i] = y_12_0_new_afterlabel_test[i]
        y_pred[i] = log_14_0.predict(X_12_0_new_afterlabel_test[i])
        proba_test[i] = log_14_0.predict_proba(X_12_0_new_afterlabel_test[i])               
        
        y_1[i] = y_12_1_new_afterlabel_test[i]
        y_pred_1[i] = log_14_1.predict(X_12_1_new_afterlabel_test[i])
        proba_test_1[i] = log_14_1.predict_proba(X_12_1_new_afterlabel_test[i])            
    if i == 5:
        y[i] = y_12_0_new_afterlabel_test[i]
        y_pred[i] = log_15_0.predict(X_12_0_new_afterlabel_test[i])
        proba_test[i] = log_15_0.predict_proba(X_12_0_new_afterlabel_test[i])                 
        
        y_1[i] = y_12_1_new_afterlabel_test[i]
        y_pred_1[i] = log_15_1.predict(X_12_1_new_afterlabel_test[i])
        proba_test_1[i] = log_15_1.predict_proba(X_12_1_new_afterlabel_test[i])        
    if i == 6:
        y[i] = y_12_0_new_afterlabel_test[i]
        y_pred[i] = log_16_0.predict(X_12_0_new_afterlabel_test[i])
        proba_test[i] = log_16_0.predict_proba(X_12_0_new_afterlabel_test[i])               
        
        y_1[i] = y_12_1_new_afterlabel_test[i]
        y_pred_1[i] = log_16_1.predict(X_12_1_new_afterlabel_test[i])
        proba_test_1[i] = log_16_1.predict_proba(X_12_1_new_afterlabel_test[i])       
    if i == 7:
        y[i] = y_12_0_new_afterlabel_test[i]
        y_pred[i] = log_17_0.predict(X_12_0_new_afterlabel_test[i])
        proba_test[i] = log_17_0.predict_proba(X_12_0_new_afterlabel_test[i])                      
        
        y_1[i] = y_12_1_new_afterlabel_test[i]
        y_pred_1[i] = log_17_1.predict(X_12_1_new_afterlabel_test[i])
        proba_test_1[i] = log_17_1.predict_proba(X_12_1_new_afterlabel_test[i])     
    if i == 8:
        y[i] = y_12_0_new_afterlabel_test[i]
        y_pred[i] = log_18_0.predict(X_12_0_new_afterlabel_test[i])
        proba_test[i] = log_18_0.predict_proba(X_12_0_new_afterlabel_test[i])           
        
        y_1[i] = y_12_1_new_afterlabel_test[i]
        y_pred_1[i] = log_18_1.predict(X_12_1_new_afterlabel_test[i])
        proba_test_1[i] = log_18_1.predict_proba(X_12_1_new_afterlabel_test[i])     
        
for i in range(9):
    roc_sum = [roc_curve(y[i],proba_test[i][:,1]) for i in range(9)]
    fpr_test = [roc_sum[i][0] for i in range(9)]
    tpr_test = [roc_sum[i][1] for i in range(9)]
    threshold = [roc_sum[i][2] for i in range(9)]
    roc_auc_test = [auc(fpr_test[i],tpr_test[i]) for i in range(9)] #calculate
    
    roc_sum_1 = [roc_curve(y_1[i],proba_test_1[i][:,1]) for i in range(9)]
    fpr_test_1 = [roc_sum_1[i][0] for i in range(9)]
    tpr_test_1 = [roc_sum_1[i][1] for i in range(9)]
    threshold_1 = [roc_sum_1[i][2] for i in range(9)]
    roc_auc_test_1 = [auc(fpr_test_1[i],tpr_test_1[i]) for i in range(9)]
        
data_0 = pandas.DataFrame({"Recall":[recall_score(y[i], y_pred[i]) for i in range(9)],"Precision":[precision_score(y[i], y_pred[i]) for i in range(9)],
                         "Accuracy":[accuracy_score(y[i], y_pred[i]) for i in range(9)],"Specificity":[specificity(y[i], y_pred[i]) for i in range(9)],
                        "AUC":roc_auc_test,"F1 score":[f1_score(y[i], y_pred[i]) for i in range(9)]})
data_1 = pandas.DataFrame({"Recall":[recall_score(y_1[i], y_pred_1[i]) for i in range(9)],"Precision":[precision_score(y_1[i], y_pred_1[i]) for i in range(9)],
                         "Accuracy":[accuracy_score(y_1[i], y_pred_1[i]) for i in range(9)],"Specificity":[specificity(y_1[i], y_pred_1[i]) for i in range(9)],
                         "AUC":roc_auc_test_1,"F1 score":[f1_score(y_1[i], y_pred_1[i]) for i in range(9)]})





        召回率       精准率       准确率       特异性     AUC面积       f1值
0  0.714286  0.312500  0.628571  0.607143  0.775510  0.434783
1  0.750000  0.391304  0.725806  0.720000  0.848333  0.514286
2  0.812500  0.464286  0.756757  0.741379  0.879310  0.590909
3  0.900000  0.428571  0.811594  0.796610  0.906780  0.580645
4  0.625000  0.416667  0.803922  0.837209  0.869186  0.500000
5  0.714286  0.312500  0.675000  0.666667  0.783550  0.434783
6  1.000000  0.500000  0.821429  0.782609  0.843478  0.666667
7  1.000000  0.545455  0.800000  0.736842  0.850877  0.705882
8  0.750000  0.600000  0.785714  0.800000  0.850000  0.666667
        召回率       精准率       准确率       特异性     AUC面积       f1值
0  0.800000  0.235294  0.674419  0.657895  0.805263  0.363636
1  0.833333  0.400000  0.773333  0.761905  0.888889  0.540541
2  0.727273  0.250000  0.696629  0.692308  0.708625  0.372093
3  0.666667  0.228571  0.693069  0.696629  0.753745  0.340426
4  0.666667  0.320000  0.783505  0.800000  0.721569  0.432432
5  0.800